In [1]:
# Client Setup
import boto3
from dotenv import load_dotenv
import os

load_dotenv()
region_name = os.getenv("AWS_REGION")

client = boto3.client("bedrock-runtime", region_name=region_name)
model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0"

In [2]:
# Helper functions


def add_user_message(messages, content):
    if isinstance(content, str):
        user_message = {"role": "user", "content": [{"text": content}]}
    else:
        user_message = {"role": "user", "content": content}
    messages.append(user_message)


def add_assistant_message(messages, content):
    if isinstance(content, str):
        assistant_message = {
            "role": "assistant",
            "content": [{"text": content}],
        }
    else:
        assistant_message = {"role": "assistant", "content": content}

    messages.append(assistant_message)


def chat(
    messages,
    system=None,
    temperature=1.0,
    stop_sequences=[],
    tools=None,
    tool_choice="auto",
    text_editor=None,
    thinking=False,
    thinking_budget=1024,
):
    params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "temperature": temperature,
            "stopSequences": stop_sequences,
        },
    }

    if system:
        params["system"] = [{"text": system}]

    tool_choices = {
        "auto": {"auto": {}},
        "any": {"any": {}},
    }
    if tools or text_editor:
        choice = tool_choices.get(tool_choice, {"tool": {"name": tool_choice}})
        params["toolConfig"] = {"tools": tools, "toolChoice": choice}

    additional_model_fields = {}
    if text_editor:
        additional_model_fields["tools"] = [
            {
                "type": text_editor,
                "name": "str_replace_editor",
            }
        ]

    if thinking:
        additional_model_fields["thinking"] = {
            "type": "enabled",
            "budget_tokens": thinking_budget,
        }

    params["additionalModelRequestFields"] = additional_model_fields
    response = client.converse(**params)
    parts = response["output"]["message"]["content"]

    return {
        "parts": parts,
        "stop_reason": response["stopReason"],
        "text": "\n".join([p["text"] for p in parts if "text" in p]),
    }


In [3]:
with open("./earth.pdf", "rb") as f:
    file_bytes = f.read()

messages = []

add_user_message(
    messages,
    [
        {   
            "document": {
                "format": "pdf", 
                "name": "earth", 
                "source": {"bytes": file_bytes},
            }
        },
        {"text": "Summarize this document in one sentence"},
    ],
)

response = chat(messages)

response["text"]

"Earth is the third planet from the Sun and the only known celestial body that harbors life, characterized by its liquid water oceans covering 70.8% of its surface, dynamic atmosphere, protective magnetosphere, and formation approximately 4.5 billion years ago from the early Solar System's protoplanetary disk."